In [ ]:
from analysis_function import *
from structural_statistics import *
import pickle
import json



final_good_file = "final_good_file"
final_filtering_good_file = "final_filtering_good_file"

#open file using pickle
with open(final_good_file, 'rb') as file:
    data = pickle.load(file)

with open(final_filtering_good_file, 'rb') as file:
    filtering = pickle.load(file)
    

In [ ]:
progressist = [
    "ban of targeted killing",
    "ban of the death penalty",
    "recognition of the right to abortion",
    "recognition of the right to euthanasia",
    "recognition of Palestinian state",
    "ban of mandatory military service",
    "ban of nuclear weapons",
    "mandatory sex education in schools",
    "guarantee of online teaching",
    "fight to climate change",
    "incentives for renewable energy",
    "ban of facial recognition technology",
    "incentives for AI research",
    "mandatory vaccination for children",
    "ban of animal testing",
    "incentives for organ donation",
    "ban of racial profiling",
    "incentives for immigration and asylum",
    "universal healthcare",
    "legalization of marijuana",
    "legalization of same-sex marriage",
    "legalization of surrogate motherhood",
    "programme for the reduction of the gender pay gap",
    "limitation to gun ownership",
    "holocaust remembrance mandatory in schools",
    "ban of zoos",
    "protection of endangered species",
    "organization of pride parades",
    "allowance of tattoos",
    "cohabitation of couples before marriage",
    "ban of arranged marriages",
    "US staying in NATO",
    "Germany staying in EU",
    "mandatory acceptance of mobile payments",
    "lowering university tuition fees",
    "mandatory cameras on police officers",
    "freedom of blasphemy",
    "legalization of adoption by same-sex couples"]

conservatist = [
    "allowance of targeted killing",
    "allowance of the death penalty",
    "ban of abortion",
    "ban of euthanasia",
    "non-recognition of Palestinian state",
    "mandatory military service",
    "support for nuclear weapons",
    "optional sex education in schools",
    "mandatory in-person teaching",
    "opposition to regulations for action on climate change",
    "incentives for energy from fossil fuels",
    "incentives for facial recognition technology",
    "opposition to AI research incentives",
    "optional vaccination for children",
    "allowance of animal testing",
    "opposition to organ donation incentives",
    "allowance of racial profiling",
    "support to immigration contrast and stricter asylum rules",
    "support to private healthcare",
    "ban of marijuana",
    "ban of same-sex marriage",
    "ban of surrogate motherhood",
    "increase of the gender pay gap in favor of men",
    "right to unrestricted gun ownership",
    "optional holocaust remembrance in schools",
    "support for zoos",
    "opposition to endangered species protection",
    "ban of pride parades",
    "ban of tattoos",
    "mandatory marriage before cohabitation",
    "right to arranged marriages",
    "US leaving NATO",
    "Germany leaving the EU",
    "ban of mobile payments",
    "increase in university tuition fees",
    "freedom of police officers to refuse cameras",
    "punishment for blasphemy",
    "ban of adoption by same-sex couples"]

topics = progressist + conservatist

In [ ]:
import json
clusters_mess = [[]] * len(topics)
clusters_conv = [[]] * len(topics)

for d, t in zip(data, filtering):
    item = json.loads(d)

    conv = ""
    mess = []

    for turn in item["conversation"]:
        conv = conv +  turn["message"] + "\n"
        mess.append(turn["message"])

    idx = topics.index(t["topic"])

    clusters_conv[idx] = clusters_conv[idx] + [conv]
    clusters_mess[idx] = clusters_mess[idx] + [mess]


In [ ]:
from sentence_transformers import SentenceTransformer, util

# Load a pretrained Sentence Transformer model
model = SentenceTransformer("all-MiniLM-L6-v2", device="cuda:0")

In [ ]:
import torch

def compare_conversations(conv_a, conv_b, model):
    # Compute the cosine similarity between the two embeddings
    similarities = util.pytorch_cos_sim(conv_a, conv_b)    #flatten the similarities matrix and return the top 5 values
    # Flatten the tensor
    flattened = similarities.flatten()

    # Get the top 5 values
    top5_values, _ = torch.topk(flattened, 5)  # Get the top 5 values
    #average the top 5 values
    top5_values = top5_values.mean()

    return top5_values.item()
    

In [ ]:
import torch

def compute_cosine_similarity_from_list(embeddings, cluster_labels, top_k=5):
    """
    Compute cosine similarity between clusters using a list of embeddings
    and corresponding cluster labels.
    
    Args:
        embeddings (list of torch.Tensor): List of individual embeddings (n x d).
        cluster_labels (list of int): Cluster label for each embedding.
        top_k (int): Number of top similarities to consider for each pair.
    
    Returns:
        dict: Average top_k similarities for each pair of clusters.
    """
    # Convert embeddings and labels to tensors
    embeddings = torch.stack(embeddings).to('cuda:0')
    cluster_labels = torch.tensor(cluster_labels).to(embeddings.device)

    # Normalize embeddings for cosine similarity
    embeddings = torch.nn.functional.normalize(embeddings, p=2, dim=1)

    # Compute the full cosine similarity matrix
    similarity_matrix = torch.mm(embeddings, embeddings.T)

    # Get unique cluster labels
    unique_labels = cluster_labels.unique()
    n_clusters = len(unique_labels)

    # Prepare mask to exclude within-cluster comparisons
    mask = torch.zeros_like(similarity_matrix, dtype=torch.bool)
    for label in unique_labels:
        cluster_indices = (cluster_labels == label).nonzero(as_tuple=True)[0]
        mask[cluster_indices[:, None], cluster_indices] = True  # Mask diagonal and within-cluster

    similarity_matrix.masked_fill_(mask, float('-inf'))  # Replace masked entries with -inf

    # Compute top-k similarities for each pair of clusters
    results = {}
    for i, label_i in enumerate(unique_labels):
        for j, label_j in enumerate(unique_labels[i + 1:], i + 1):
            # Get indices for the two clusters
            indices_i = (cluster_labels == label_i).nonzero(as_tuple=True)[0]
            indices_j = (cluster_labels == label_j).nonzero(as_tuple=True)[0]

            # Extract sub-matrix for the cluster pair
            sub_matrix = similarity_matrix[indices_i][:, indices_j]

            # Get top-k similarities for each row
            top_k_values, _ = torch.topk(sub_matrix, k=top_k, dim=1)

            # Average the top-k values
            avg_top_k_similarity = torch.mean(top_k_values).item()
            results[(label_i.item(), label_j.item())] = avg_top_k_similarity
            #print(f"Avg Top-{top_k} Similarity between cluster {label_i.item()} and {label_j.item()}: {avg_top_k_similarity}")

    return results

In [ ]:
embeddings = [[]] * len(topics)

for idx, cluster in enumerate(clusters_mess):
    print(idx)
    for i in range(len(cluster)):
        embeddings[idx] = embeddings[idx] + [model.encode(cluster[i])]

In [ ]:

rem_embeddings = [[]] * len(topics)
cluster_labels = [[]] * len(topics)

for idx, emb in enumerate(embeddings):
    rem_emb = []
    cl_labels = []
    for i, e in enumerate(emb):
        for ten in e:
            rem_emb = rem_emb + [torch.Tensor(ten)]
            cl_labels = cl_labels + [i]
            
    rem_embeddings[idx] = rem_emb
    cluster_labels[idx] = cl_labels
    


In [ ]:

results = []
count = 0
for emb, lab in zip(rem_embeddings, cluster_labels):
    count = count + 1
    print(count)
    results.append(compute_cosine_similarity_from_list(emb, lab, top_k=5))

In [ ]:
res_cluster = []
for cluster in results:
    res = []
    for key in cluster:
        res.append(cluster[key])

    res_cluster.append(sum(res)/len(res))

print(sum(res_cluster)/len(res_cluster))